In [1]:
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely.wkb 
from shapely.ops import cascaded_union
from shapely.geometry import Polygon, Point, MultiPolygon
from tqdm import tqdm

# Protected areas percentage

### Read Global Facets table

In [7]:
path = '/Users/ikersanchez/Vizzuality/PROIEKTUAK/Half-Earth/work/data/'

In [35]:
df = gpd.read_file(path+'global_facets_attr_pressures_all/global_facets_attr_pressures.shp')
df.columns = map(str.lower, df.columns)

### Read Terrestrial and Marine grids

We read the grids from the carto account

In [3]:
account = 'half-earth'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

**Terrestrial grid**

In [5]:
query =("SELECT cartodb_id, cell_id, the_geom FROM terrestrial_grid")

sql = {"q": query}
r = requests.get(urlCarto, params=sql)

data = r.json()

terrestrial = gpd.GeoDataFrame(data.get("rows"))
# Change geometry from WKB to WKT format
terrestrial['geometry'] = terrestrial.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
terrestrial.drop(columns='the_geom', inplace=True)
terrestrial.sort_values(by=['cartodb_id'], inplace=True)

**Marine grid**

In [6]:
query =("SELECT cartodb_id, cell_id, the_geom FROM marine_grid")

sql = {"q": query}
r = requests.get(urlCarto, params=sql)

data = r.json()

marine = gpd.GeoDataFrame(data.get("rows"))
# Change geometry from WKB to WKT format
marine['geometry'] = marine.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
marine.drop(columns='the_geom', inplace=True)
marine.sort_values(by=['cartodb_id'], inplace=True)

### Read Protected Areas

In [9]:
protected_areas = gpd.read_file(path+'wdpa_protected_areas/wdpa_protected_areas.shp')

**Strict**

In [10]:
strict = protected_areas[(protected_areas['iucn_cat'] == 'Ia') | 
                         (protected_areas['iucn_cat'] == 'Ib') |
                         (protected_areas['iucn_cat'] == 'II') |
                         (protected_areas['iucn_cat'] == 'III')
                        ][['cartodb_id', 'geometry']]

**Biosphere**

In [21]:
biosphere = protected_areas[protected_areas['desig_eng'].str.contains('Biosphere', regex=False)][['cartodb_id', 'geometry']]

**Other**

In [31]:
other = strict = protected_areas[(protected_areas['iucn_cat'] != 'Ia') & 
                         (protected_areas['iucn_cat'] != 'Ib') &
                         (protected_areas['iucn_cat'] != 'II') &
                         (protected_areas['iucn_cat'] != 'III') &
                         (~protected_areas['desig_eng'].str.contains('Aborig', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Indigen', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Commun', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Conservanc', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Local', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Region', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Trust', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Conservator', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Private', regex=False)) &
                         (~protected_areas['desig_eng'].str.contains('Nature Center', regex=False)) 
                        ][['cartodb_id', 'geometry']]

**Conservation**

In [33]:
conservation = gpd.read_file(path+'community_conservation_areas/community_conservation_areas.shp')

### Percentege of protected areas inside each cell

We use geopandas’s R-tree spatial index to find which protected area lie within each grid cells

In [36]:
def rtree_intersect(gdf, grid):
    percentage = []
    sindex = gdf.sindex
    # we iterate over the cells
    for n, cell in enumerate(tqdm(grid.geometry)):
        # protected areas that intersect with the cell
        possible_matches_index = list(sindex.intersection(cell.bounds))
        possible_matches = gdf.iloc[possible_matches_index]
        # intersection between the protected areas and the cell
        precise_matches = possible_matches.intersection(cell)
        # percentage of the cell covered by protected areas
        percentage.append(cascaded_union(precise_matches).area/cell.area*100) 
    return percentage

In [ ]:
# DataFrames to save the outputs
marine_per = marine[['cell_id', 'geometry']]
terrestrial_per = terrestrial[['cell_id', 'geometry']]

# Dictionaries with the different grids and protected areas
grids = {"terrestrial":terrestrial, "marine":marine}
areas = {"strict":strict, "biosphere":biosphere, "conservation":conservation, "other":other}

for grid_key in grids.keys():
    for area_key in areas.keys():
        print(f"For {grid_key}, {area_key}")
        percentage = rtree_intersect(areas[area_key], grids[grid_key])
        if grid_key is "terrestrial":
            terrestrial_per[area_key] = percentage
            terrestrial_per[area_key] = terrestrial_per[area_key].map('{:3.1f}'.format).apply(pd.to_numeric)
        else:
            marine_per[area_key] = percentage
            marine_per[area_key] = marine_per[area_key].map('{:3.1f}'.format).apply(pd.to_numeric)

For terrestrial, strict


 67%|██████▋   | 8517/12670 [21:27<69:43:10, 60.44s/it]

**Add percentage columns to table**

In [ ]:
terrestrial_per.drop(columns=['geometry'], inplace= True)
marine_per.drop(columns=['geometry'], inplace= True)
per = pd.concat([terrestrial_per,marine_per])
df_new = pd.merge(df, per, how='left', on=['cell_id'])

**Save new table**

In [ ]:
df_new.to_file(path+'global_facets_attr_pressures_vizz/global_facets_attr_pressures_vizz.shp')